In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 종목명과 종목코드 딕셔너리
stocks = {
    "엠케이전자": "033160",
    "피델릭스": "032580",
    "GRT": "900290",
    "플리토": "300080",
    "라온테크": "232680",
    "인스웨이브시스템즈": "419260",
    "코난테크놀로지": "402030",
    "솔트룩스": "304100",
    "바이브컴퍼니": "301300",
    "마음AI": "377480",
    "오브젠": "417860",
    "모니터랩": "434480",
}

# 요청 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# 데이터 저장용 딕셔너리 (날짜를 키로 사용)
data_dict = {}

# 모든 종목 데이터 수집
for stock_name, stock_code in stocks.items():
    for page in range(1, 2):  # 1부터 4페이지까지 조회
        url = f"https://finance.naver.com/item/frgn.naver?code={stock_code}&page={page}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            rows = soup.select("table.type2 tr")

            for row in rows:
                cols = row.select("td")  # 테이블의 각 열(td) 추출
                
                if len(cols) >= 9:  # 데이터가 존재하는 행인지 확인
                    date = cols[0].text.strip()  # 날짜
                    change_rate = cols[3].text.strip().replace('%', '')  # 변동률

                    if date and change_rate:  # 유효한 데이터만 추가
                        if date not in data_dict:
                            data_dict[date] = {}
                        
                        data_dict[date][f"{stock_name}"] = change_rate
        else:
            print(f"{stock_name}의 페이지 {page}를 불러오지 못했습니다. 상태 코드:", response.status_code)

# 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame.from_dict(data_dict, orient='index')
df.index.name = '날짜'

# 날짜 순 정렬
df = df.sort_index(ascending=True)

# 엑셀로 저장
file_name = "딥시크크.xlsx"
df.to_excel(file_name)

print(f"모든 종목의 변동률 데이터가 '{file_name}'로 저장되었습니다.")


모든 종목의 변동률 데이터가 '딥시크크.xlsx'로 저장되었습니다.
